In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 19:03:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# df_fake = spark.read.option("header", "true").csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/fake.csv', sep=',', multiLine=True)
# df_true = spark.read.option("header", "true").csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/true.csv', sep=',', multiLine=True)

In [10]:
# Funtion for conver Pandas Dataframe to Spark Dataframe
import pandas as pd
from pyspark.sql.types import StringType, StructField, StructType
def read_data(path):
  schema= StructType(
      [StructField('title',StringType(),True),
      StructField('text',StringType(),True),
      StructField('subject',StringType(),True),
      StructField('date',StringType(),True)])
  pd_df= pd.read_csv(path)
  sp_df= spark.createDataFrame(pd_df, schema= schema)
  return sp_df

In [11]:
df_fake= read_data("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/fake.csv")
df_true= read_data("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/true.csv")

In [13]:
# Concatenate 2 data sets into one and shuffle data set
from pyspark.sql.functions import lit, rand
df_concat= df_true.withColumn('fake', lit(0)).union(df_fake.withColumn('fake', lit(1))).orderBy(rand())

In [14]:
df_concat

22/12/04 19:04:45 WARN TaskSetManager: Stage 0 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:46 WARN TaskSetManager: Stage 1 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,text,subject,date,fake
WILL TAYLOR SWIFT...,Misogyny is ingr...,left-news,"May 20, 2015",1
Highlights of Reu...,WASHINGTON (Reute...,politicsNews,"April 28, 2017",0
Obama makes new p...,WASHINGTON (Reute...,politicsNews,"January 26, 2016",0
Trump's whirlwind...,"LYON, France (Reu...",politicsNews,"February 5, 2017",0
WATCH: FORMER DNC...,Former Democratic...,politics,"Aug 26, 2017",1
Legal Fears Push ...,21st Century Wire...,US_News,"July 23, 2017",1
LT GEN MCINERNEY’...,,politics,"Jan 13, 2017",1
WATCH MAGA RALLY ...,PRESIDENT TRUMP I...,left-news,"Aug 22, 2017",1
Billion Dollar C...,You may not be fa...,News,"September 25, 2017",1
Brazil government...,BRASILIA (Reuters...,worldnews,"October 20, 2017",0


In [15]:
# # Remove the unwanted coloumn

# df_concat =  df_concat.drop(df_concat["date"])
# df_concat

In [16]:
# # Remove the title for analysis

# df_concat =  df_concat.drop(df_concat["title"])
# df_concat

In [17]:
# df_concat.select('subject').distinct().show()

In [18]:
# In order to avaoid the confusion convert everything to lowercase
from pyspark.sql.functions import col, split, lower, regexp_replace, length

df_concat = df_concat\
.withColumn("text", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.where(length("text")>0)

df_concat

22/12/04 19:04:49 WARN TaskSetManager: Stage 2 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:50 WARN TaskSetManager: Stage 3 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,text,subject,date,fake
WILL TAYLOR SWIFT...,misogyny is ingr...,left-news,"May 20, 2015",1
Highlights of Reu...,washington reute...,politicsNews,"April 28, 2017",0
Obama makes new p...,washington reute...,politicsNews,"January 26, 2016",0
Trump's whirlwind...,lyon france reu...,politicsNews,"February 5, 2017",0
WATCH: FORMER DNC...,former democratic...,politics,"Aug 26, 2017",1
Legal Fears Push ...,21st century wire...,US_News,"July 23, 2017",1
LT GEN MCINERNEY’...,,politics,"Jan 13, 2017",1
WATCH MAGA RALLY ...,president trump i...,left-news,"Aug 22, 2017",1
Billion Dollar C...,you may not be fa...,News,"September 25, 2017",1
Brazil government...,brasilia reuters...,worldnews,"October 20, 2017",0


In [19]:
# remove stop words from it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import pyspark.sql.functions as F

df_concat = df_concat.withColumn('words',F.split(F.col('text'),' '))
df_concat

22/12/04 19:04:53 WARN TaskSetManager: Stage 4 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:53 WARN TaskSetManager: Stage 5 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:54 WARN TaskSetManager: Stage 8 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:55 WARN TaskSetManager: Stage 9 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,text,subject,date,fake,words
WILL TAYLOR SWIFT...,misogyny is ingr...,left-news,"May 20, 2015",1,"[, misogyny, is, ..."
Highlights of Reu...,washington reute...,politicsNews,"April 28, 2017",0,"[washington, , re..."
Obama makes new p...,washington reute...,politicsNews,"January 26, 2016",0,"[washington, , re..."
Trump's whirlwind...,lyon france reu...,politicsNews,"February 5, 2017",0,"[lyon, , france, ..."
WATCH: FORMER DNC...,former democratic...,politics,"Aug 26, 2017",1,"[former, democrat..."
Legal Fears Push ...,21st century wire...,US_News,"July 23, 2017",1,"[21st, century, w..."
LT GEN MCINERNEY’...,,politics,"Jan 13, 2017",1,"[, ]"
WATCH MAGA RALLY ...,president trump i...,left-news,"Aug 22, 2017",1,"[president, trump..."
Billion Dollar C...,you may not be fa...,News,"September 25, 2017",1,"[you, may, not, b..."
Brazil government...,brasilia reuters...,worldnews,"October 20, 2017",0,"[brasilia, , reut..."


In [21]:
from pyspark.ml.feature import StopWordsRemover 

stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                   outputCol="words without stop").setStopWords(stop)

In [22]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df_concat)

In [23]:
df_concat = pipelineFitRemoveStopWords.transform(df_concat)

In [24]:
df_concat

22/12/04 19:04:57 WARN TaskSetManager: Stage 12 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:57 WARN TaskSetManager: Stage 13 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:58 WARN TaskSetManager: Stage 16 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:04:59 WARN TaskSetManager: Stage 17 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,text,subject,date,fake,words,words without stop
WILL TAYLOR SWIFT...,misogyny is ingr...,left-news,"May 20, 2015",1,"[, misogyny, is, ...","[, misogyny, ingr..."
Highlights of Reu...,washington reute...,politicsNews,"April 28, 2017",0,"[washington, , re...","[washington, , re..."
Obama makes new p...,washington reute...,politicsNews,"January 26, 2016",0,"[washington, , re...","[washington, , re..."
Trump's whirlwind...,lyon france reu...,politicsNews,"February 5, 2017",0,"[lyon, , france, ...","[lyon, , france, ..."
WATCH: FORMER DNC...,former democratic...,politics,"Aug 26, 2017",1,"[former, democrat...","[former, democrat..."
Legal Fears Push ...,21st century wire...,US_News,"July 23, 2017",1,"[21st, century, w...","[21st, century, w..."
LT GEN MCINERNEY’...,,politics,"Jan 13, 2017",1,"[, ]","[, ]"
WATCH MAGA RALLY ...,president trump i...,left-news,"Aug 22, 2017",1,"[president, trump...","[president, trump..."
Billion Dollar C...,you may not be fa...,News,"September 25, 2017",1,"[you, may, not, b...","[may, familiar, n..."
Brazil government...,brasilia reuters...,worldnews,"October 20, 2017",0,"[brasilia, , reut...","[brasilia, , reut..."


In [25]:
# convert array of strings to strings
from pyspark.sql.functions import concat_ws

df_concat = df_concat.withColumn("text1", concat_ws(" ", df_concat["words without stop"]))
df_concat

22/12/04 19:05:01 WARN TaskSetManager: Stage 20 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:01 WARN TaskSetManager: Stage 21 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:02 WARN TaskSetManager: Stage 24 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:03 WARN TaskSetManager: Stage 25 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,text,subject,date,fake,words,words without stop,text1
WILL TAYLOR SWIFT...,misogyny is ingr...,left-news,"May 20, 2015",1,"[, misogyny, is, ...","[, misogyny, ingr...",misogyny ingrain...
Highlights of Reu...,washington reute...,politicsNews,"April 28, 2017",0,"[washington, , re...","[washington, , re...",washington reute...
Obama makes new p...,washington reute...,politicsNews,"January 26, 2016",0,"[washington, , re...","[washington, , re...",washington reute...
Trump's whirlwind...,lyon france reu...,politicsNews,"February 5, 2017",0,"[lyon, , france, ...","[lyon, , france, ...",lyon france reu...
WATCH: FORMER DNC...,former democratic...,politics,"Aug 26, 2017",1,"[former, democrat...","[former, democrat...",former democratic...
Legal Fears Push ...,21st century wire...,US_News,"July 23, 2017",1,"[21st, century, w...","[21st, century, w...",21st century wire...
LT GEN MCINERNEY’...,,politics,"Jan 13, 2017",1,"[, ]","[, ]",
WATCH MAGA RALLY ...,president trump i...,left-news,"Aug 22, 2017",1,"[president, trump...","[president, trump...",president trump p...
Billion Dollar C...,you may not be fa...,News,"September 25, 2017",1,"[you, may, not, b...","[may, familiar, n...",may familiar name...
Brazil government...,brasilia reuters...,worldnews,"October 20, 2017",0,"[brasilia, , reut...","[brasilia, , reut...",brasilia reuters...


In [26]:
df_concat = df_concat.drop("words","words without stop")

df_concat

22/12/04 19:05:04 WARN TaskSetManager: Stage 28 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:05 WARN TaskSetManager: Stage 29 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:06 WARN TaskSetManager: Stage 32 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:07 WARN TaskSetManager: Stage 33 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,text,subject,date,fake,text1
WILL TAYLOR SWIFT...,misogyny is ingr...,left-news,"May 20, 2015",1,misogyny ingrain...
Highlights of Reu...,washington reute...,politicsNews,"April 28, 2017",0,washington reute...
Obama makes new p...,washington reute...,politicsNews,"January 26, 2016",0,washington reute...
Trump's whirlwind...,lyon france reu...,politicsNews,"February 5, 2017",0,lyon france reu...
WATCH: FORMER DNC...,former democratic...,politics,"Aug 26, 2017",1,former democratic...
Legal Fears Push ...,21st century wire...,US_News,"July 23, 2017",1,21st century wire...
LT GEN MCINERNEY’...,,politics,"Jan 13, 2017",1,
WATCH MAGA RALLY ...,president trump i...,left-news,"Aug 22, 2017",1,president trump p...
Billion Dollar C...,you may not be fa...,News,"September 25, 2017",1,may familiar name...
Brazil government...,brasilia reuters...,worldnews,"October 20, 2017",0,brasilia reuters...


In [27]:
df_concat = df_concat.drop("text")

df_concat

22/12/04 19:05:08 WARN TaskSetManager: Stage 36 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:09 WARN TaskSetManager: Stage 37 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,subject,date,fake,text1
WILL TAYLOR SWIFT...,left-news,"May 20, 2015",1,misogyny ingrain...
Highlights of Reu...,politicsNews,"April 28, 2017",0,washington reute...
Obama makes new p...,politicsNews,"January 26, 2016",0,washington reute...
Trump's whirlwind...,politicsNews,"February 5, 2017",0,lyon france reu...
WATCH: FORMER DNC...,politics,"Aug 26, 2017",1,former democratic...
Legal Fears Push ...,US_News,"July 23, 2017",1,21st century wire...
LT GEN MCINERNEY’...,politics,"Jan 13, 2017",1,
WATCH MAGA RALLY ...,left-news,"Aug 22, 2017",1,president trump p...
Billion Dollar C...,News,"September 25, 2017",1,may familiar name...
Brazil government...,worldnews,"October 20, 2017",0,brasilia reuters...


In [28]:
df_concat = df_concat.withColumnRenamed("text1","text")

df_concat

22/12/04 19:05:10 WARN TaskSetManager: Stage 38 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:10 WARN TaskSetManager: Stage 39 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,subject,date,fake,text
WILL TAYLOR SWIFT...,left-news,"May 20, 2015",1,misogyny ingrain...
Highlights of Reu...,politicsNews,"April 28, 2017",0,washington reute...
Obama makes new p...,politicsNews,"January 26, 2016",0,washington reute...
Trump's whirlwind...,politicsNews,"February 5, 2017",0,lyon france reu...
WATCH: FORMER DNC...,politics,"Aug 26, 2017",1,former democratic...
Legal Fears Push ...,US_News,"July 23, 2017",1,21st century wire...
LT GEN MCINERNEY’...,politics,"Jan 13, 2017",1,
WATCH MAGA RALLY ...,left-news,"Aug 22, 2017",1,president trump p...
Billion Dollar C...,News,"September 25, 2017",1,may familiar name...
Brazil government...,worldnews,"October 20, 2017",0,brasilia reuters...


In [29]:
# most frequent words in fake news

df_fake = df_concat.filter(df_concat["fake"] == 1)

df_fake_text = df_fake.select("text")

df_fake_text.show()

22/12/04 19:05:11 WARN TaskSetManager: Stage 40 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|                text|
+--------------------+
| misogyny ingrain...|
|former democratic...|
|21st century wire...|
|                    |
|president trump p...|
|may familiar name...|
|president trump l...|
|president trump s...|
|reporter model  l...|
|donald trump long...|
|another member sw...|
|congressman al gr...|
| salt police  com...|
|american liberal ...|
|conservatives rep...|
|donald trump beha...|
|hillary clinton c...|
|thursday visit me...|
|another staged ph...|
|muslims faith mus...|
+--------------------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_fake.orderBy(col("count").desc()).show()

22/12/04 19:05:12 WARN TaskSetManager: Stage 41 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:05:15 WARN TaskSetManager: Stage 42 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+---------+-------+
|     word|  count|
+---------+-------+
|         |1358755|
|    trump|  79352|
|     said|  33763|
|president|  27715|
|   people|  26570|
|      one|  24531|
|    would|  23562|
|  clinton|  19173|
|    obama|  18797|
|     like|  18098|
|   donald|  17681|
|     also|  15403|
|       us|  14822|
|     news|  14629|
|      new|  14394|
|  hillary|  14122|
|     even|  14011|
|     time|  13870|
|    state|  13465|
|    white|  13189|
+---------+-------+
only showing top 20 rows



In [31]:
# The highest number of news

df = df_concat.groupBy('subject').count()
df.orderBy(col("count").desc()).show(truncate=False)

22/12/04 19:05:20 WARN TaskSetManager: Stage 48 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+---------------+-----+
|subject        |count|
+---------------+-----+
|politicsNews   |11272|
|worldnews      |10145|
|News           |9050 |
|politics       |6841 |
|left-news      |4459 |
|Government News|1570 |
|US_News        |783  |
|Middle-east    |778  |
+---------------+-----+



In [32]:
from pyspark.ml.feature import SQLTransformer, RegexTokenizer, StopWordsRemover, CountVectorizer, Imputer, IDF
from pyspark.ml.feature import StringIndexer, VectorAssembler
StopWordsRemover.loadDefaultStopWords('english')

# 0. Extract tokens from title
title_tokenizer= RegexTokenizer(inputCol= 'title', outputCol= 'title_words',
                                pattern= '\\W', toLowercase= True)
# 1. Remove stop words from title
title_sw_remover= StopWordsRemover(inputCol= 'title_words', outputCol= 'title_sw_removed')
# 2. Compute Term frequency from title
title_count_vectorizer= CountVectorizer(inputCol= 'title_sw_removed', outputCol= 'tf_title')
# 3. Compute Term frequency-inverse document frequency from title
title_tfidf= IDF(inputCol= 'tf_title', outputCol= 'tf_idf_title')
# 4. Extract tokens from text
text_tokenizer= RegexTokenizer(inputCol= 'text', outputCol= 'text_words',
                                pattern= '\\W', toLowercase= True)
# 5. Remove stop words from text
text_sw_remover= StopWordsRemover(inputCol= 'text_words', outputCol= 'text_sw_removed')
# 6. Compute Term frequency from text
text_count_vectorizer= CountVectorizer(inputCol= 'text_sw_removed', outputCol= 'tf_text')
# 7. Compute Term frequency-inverse document frequency text
text_tfidf= IDF(inputCol= 'tf_text', outputCol= 'tf_idf_text')
# 8. StringIndexer subject
subject_str_indexer= StringIndexer(inputCol= 'subject', outputCol= 'subject_idx')
# 9. VectorAssembler
vec_assembler= VectorAssembler(inputCols=['tf_idf_title', 'tf_idf_text', 'subject_idx'], outputCol= 'features')

In [33]:
from pyspark.ml.classification import RandomForestClassifier
# 10 Random Forest Classifier
rf= RandomForestClassifier(featuresCol= 'features', labelCol= 'fake', predictionCol= 'fake_predict', maxDepth= 7, numTrees= 20)

rf

RandomForestClassifier_8fcfd1c67b8b

In [34]:
from pyspark.ml import Pipeline
rf_pipe= Pipeline(stages=[title_tokenizer, # 0
                title_sw_remover, # 1
                title_count_vectorizer, # 2
                title_tfidf, # 3
                text_tokenizer, # 4
                text_sw_remover, # 5
                text_count_vectorizer, # 6
                text_tfidf, # 7
                subject_str_indexer, # 8
                vec_assembler, # 9
                rf]) # 10 model

In [35]:
train, test= df_concat.randomSplit([0.8, 0.2])

In [36]:
rf_model= rf_pipe.fit(train)

22/12/04 19:06:07 WARN TaskSetManager: Stage 51 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:06:10 WARN TaskSetManager: Stage 55 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:06:13 WARN TaskSetManager: Stage 57 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:06:17 WARN DAGScheduler: Broadcasting large task binary with size 1282.1 KiB
22/12/04 19:06:17 WARN TaskSetManager: Stage 61 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:06:20 WARN DAGScheduler: Broadcasting large task binary with size 1283.1 KiB
22/12/04 19:06:20 WARN TaskSetManager: Stage 63 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


...

22/12/04 19:06:49 WARN MemoryStore: Not enough space to cache rdd_265_3 in memory! (computed 135.9 MiB so far)
22/12/04 19:06:49 WARN BlockManager: Persisting block rdd_265_3 to disk instead.
22/12/04 19:06:49 WARN MemoryStore: Not enough space to cache rdd_265_4 in memory! (computed 208.8 MiB so far)
22/12/04 19:06:49 WARN BlockManager: Persisting block rdd_265_4 to disk instead.
22/12/04 19:06:49 WARN MemoryStore: Not enough space to cache rdd_265_5 in memory! (computed 314.0 MiB so far)
22/12/04 19:06:49 WARN BlockManager: Persisting block rdd_265_5 to disk instead.
22/12/04 19:06:49 WARN MemoryStore: Not enough space to cache rdd_265_7 in memory! (computed 314.0 MiB so far)
22/12/04 19:06:49 WARN BlockManager: Persisting block rdd_265_7 to disk instead.
22/12/04 19:06:49 WARN MemoryStore: Not enough space to cache rdd_265_2 in memory! (computed 314.0 MiB so far)
22/12/04 19:06:49 WARN BlockManager: Persisting block rdd_265_2 to disk instead.
22/12/04 19:06:51 WARN MemoryStore: Not 

22/12/04 19:06:56 WARN MemoryStore: Not enough space to cache rdd_265_3 in memory! (computed 208.8 MiB so far)
22/12/04 19:06:56 WARN MemoryStore: Not enough space to cache rdd_265_7 in memory! (computed 314.0 MiB so far)
22/12/04 19:06:56 WARN MemoryStore: Not enough space to cache rdd_265_4 in memory! (computed 314.0 MiB so far)
[189.232s][warning][gc,alloc] Executor task launch worker for task 8.0 in stage 73.0 (TID 697): Retried waiting for GCLocker too often allocating 65282 words
22/12/04 19:06:57 WARN BlockManager: Block rdd_265_8 could not be removed as it was not found on disk or in memory
[189.241s][warning][gc,alloc] Executor task launch worker for task 6.0 in stage 73.0 (TID 695): Retried waiting for GCLocker too often allocating 65282 words
[189.241s][warning][gc,alloc] Executor task launch worker for task 7.0 in stage 73.0 (TID 696): Retried waiting for GCLocker too often allocating 65282 words
22/12/04 19:06:57 ERROR Executor: Exception in task 8.0 in stage 73.0 (TID 697

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49446)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/s6/3zpv69vn7939njdmq8qdwbrm0000gn/T/ipykernel_1036/1782348987.py", line 1, in <cell line: 1>
    rf_model= rf_pipe.fit(train)
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/ml/pipeline.py", line 134, in _fit
    model = stage.fit(dataset)
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/py

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


## Fake News Detection (You Tube)

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [38]:
df_fake = pd.read_csv("./datasets/kaggle/Fake.csv")
df_true = pd.read_csv("./datasets/kaggle/True.csv")

In [39]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [40]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [41]:
df_fake.shape, df_true.shape

((23481, 4), (21417, 4))

In [42]:
df_fake["class"] = 0
df_true["class"] = 1

In [43]:
df_fake_manula_tesing = df_fake.tail(10)
for i in range(23480, 23470, -1):
    df_fake.drop([i],axis=0, inplace=True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21406, -1):
    df_true.drop([i],axis=0, inplace=True)

In [44]:
df_manual_testing = pd.concat([df_fake_manula_tesing, df_true_manual_testing], axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [45]:
df_merge = pd.concat([df_fake,df_true], axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [46]:
df = df_merge.drop(["title","subject","date"],axis=1)
df

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
21402,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,1
21403,WASHINGTON (Reuters) - The United States sugge...,1
21404,WASHINGTON (Reuters) - The United States has d...,1
21405,ISLAMABAD (Reuters) - Outlining a new strategy...,1


In [47]:
#shuffle
df = df.sample(frac=1)
df

,text,class
19935,,0
2136,NEW YORK/WASHINGTON (Reuters) - Several U.S. c...,1
21498,Disrupting people when they re trying to have ...,0
6051,It would make sense why Trump claimed wages ar...,0
13232,Crooked lying Hillary,0
...,...,...
1653,WASHINGTON (Reuters) - Senate Republican leade...,1
13327,SEOUL/MOSCOW (Reuters) - North Korea s latest ...,1
272,"Clearly, Donald Trump intends to do absolutely...",0
11893,,0


In [48]:
#to check null values
df.isnull().sum()

text     0
class    0
dtype: int64

In [49]:
df['text'] = df['text'].apply(lambda x: x.lower())
df.head()

,text,class
19935,,0
2136,new york/washington (reuters) - several u.s. c...,1
21498,disrupting people when they re trying to have ...,0
6051,it would make sense why trump claimed wages ar...,0
13232,crooked lying hillary,0


In [50]:
# Remove punctuation

import string

def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

df['text'] = df['text'].apply(punctuation_removal)
df

,text,class
19935,,0
2136,new yorkwashington reuters several us chief e...,1
21498,disrupting people when they re trying to have ...,0
6051,it would make sense why trump claimed wages ar...,0
13232,crooked lying hillary,0
...,...,...
1653,washington reuters senate republican leader m...,1
13327,seoulmoscow reuters north korea s latest miss...,1
272,clearly donald trump intends to do absolutely ...,0
11893,,0


In [51]:
# Removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
df

,text,class
19935,,0
2136,new yorkwashington reuters several us chief ex...,1
21498,disrupting people trying dinner came end ted c...,0
6051,would make sense trump claimed wages high supp...,0
13232,crooked lying hillary,0
...,...,...
1653,washington reuters senate republican leader mi...,1
13327,seoulmoscow reuters north korea latest missile...,1
272,clearly donald trump intends absolutely nothin...,0
11893,,0


In [53]:
x = df["text"]
y= df["class"]
print(type(x))

<class 'pandas.core.series.Series'>


In [54]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.25)

In [55]:
#vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

Logistic regression

In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [59]:
LR.score(xv_test, y_test)

0.9901069518716578

In [60]:
pred_LR = LR.predict(xv_test)

In [61]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5871
           1       0.99      0.99      0.99      5349

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



Decidion Tree

In [240]:
from sklearn.tree import DecisionTreeClassifier

In [241]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [243]:
DT.score(xv_test, y_test)

0.9977718360071302

In [244]:
pred_DT = DT.predict(xv_test)

In [245]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5927
           1       1.00      1.00      1.00      5293

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



Gradient boosting classifier

In [246]:
from sklearn.ensemble import GradientBoostingClassifier

In [247]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [248]:
pred_gbc = GBC.predict(xv_test)

In [249]:
GBC.score(xv_test, y_test)

0.9963458110516934

In [250]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5927
           1       0.99      1.00      1.00      5293

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



Random Forest Classifier

In [252]:
from sklearn.ensemble import RandomForestClassifier

In [253]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [254]:
pred_rfc = RFC.predict(xv_test)

In [255]:
RFC.score(xv_test, y_test)

0.9942959001782531

In [256]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5927
           1       0.99      1.00      0.99      5293

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [62]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [313]:
def output_lable(n):
    print(type(n))
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [315]:
news = str(input())
manual_testing(news)

 By Dady Chery and Gilbert MercierAll writers with a desire to rattle people out of their torpor occasionally wonder if it is worthwhile to continue to try to raise their voices over the din of lies and distractions.More and more for us, such thoughts are occasioned, not by the mainstream, which predictably treats all the pronouncements from the powerful as being newsworthy, but by the fake left, which lobotomizes most of whom it touches. The increasing sophistication of this group and its rate of expansion are astounding. Its purpose is to annihilate and replace the real left, and it is making great strides in this regard.The real left includes those who, like Dr. Martin Luther King Jr. (MLK), came to realize that quantum leaps are the way to proceed, rather than tiny incremental change. The real left does not curry favor or approval from those in power; it challenges the social status quo and respects the inherent right of an injured and indignant population to rebel against authorit

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GBC Prediction: Not A Fake News 
RFC Prediction: Not A Fake News
